In [1]:
import pandas as pd
import numpy as np

<h2 style='color:#00A'>O código adiante obtém os arquivos csv, por tipo (rat, rat_evetivo, rat_viatura e rat_produtividade) e os ordena por data</h2>

In [2]:
import funcs
import itertools

df_files = pd.read_sql_table('tbl_imported_files', 'sqlite:///gdo.db')

files_names = funcs.get_files_names()
files_names = list(itertools.chain.from_iterable(files_names))

files_not_imported = [file_name for file_name in files_names if file_name not in df_files['0'].values ]
files_not_imported

['REDS_RAT_20200411_20200420.csv',
 'REDS_RAT_VIATURAS_20200411_20200420.csv',
 'REDS_RAT_EFETIVOS_20200411_20200420.csv',
 'REDS_RAT_Produtividade_20200411_20200420.csv']

In [33]:
df = pd.DataFrame({
    'A':[1,2,3],
    'B':[4,5,6]
})


False

In [18]:
# pd.DataFrame(rats_files).to_sql('tbl_imported_files','sqlite:///gdo.db',if_exists='append')
# pd.DataFrame(rats_viatura_files).to_sql('tbl_imported_files','sqlite:///gdo.db',if_exists='append')
# pd.DataFrame(rats_efetivo_files).to_sql('tbl_imported_files','sqlite:///gdo.db',if_exists='append')
# pd.DataFrame(rats_produtividade_files).to_sql('tbl_imported_files','sqlite:///gdo.db',if_exists='append')

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: tbl_imported_files.0
[SQL: INSERT INTO tbl_imported_files ("index", "0") VALUES (?, ?)]
[parameters: ((0, 'REDS_RAT_20190101_20190110.csv'), (1, 'REDS_RAT_20190111_20190120.csv'), (2, 'REDS_RAT_20190121_20190131.csv'), (3, 'REDS_RAT_20190201_20190210.csv'), (4, 'REDS_RAT_20190211_20190220.csv'), (5, 'REDS_RAT_20190221_20190228.csv'), (6, 'REDS_RAT_20190301_20190310.csv'), (7, 'REDS_RAT_20190311_20190320.csv')  ... displaying 10 of 47 total bound parameter sets ...  (45, 'REDS_RAT_20200401_20200410.csv'), (46, 'REDS_RAT_20200411_20200420.csv'))]
(Background on this error at: http://sqlalche.me/e/gkpj)

<h2 style="color:#0A0;margin:10px ">Importação, para o Banco de Dados, de todos os arquivos tipo RAT</h2>
<h3 style="color:#00A; margin:10px">Importa somente os dados do 23º BPM</h3>

<ol style="color:#222">
    <li>Lê o primeiro arquivo da lista e o passa para um DataFrame (df1)</li>
    <li>Lê o segundo arquivo da lista e o passa para outro DataFrame (df2)</li>
    <li>Concatena os dois DataFrames em um terceiro (df_acum)</li>
    <li>Itera sobre os demais arquivos, lendo cada um deles, aplicando strip(), a função de filtro do 23º BPM e depois o adiciona ao DataFrame df_acum</li>
    <li>Retira as duplicatas, pelo campo 'RAT.NUM_ATIVIDADE'</li>
    <li>Adiciona colunas data/hora início e término, tempo em datetime e tempo em inteiro </li>
    <li>Aplica .upper() nas colunas usadas para classificação.</li>
    <li>Classifica por Setores</li>
    <li>Classifica por Cias</li>
    <li>Insere os dados de df_acum no banco de dados gdo.db: (tabela tbl_rat)</li>

</ol>

In [26]:
# df_classif = pd.read_csv('files/classificadores.csv')
# df_classif = df_classif.apply(lambda col:col.str.upper())
# df_classif.set_index( df_classif['MUNICIPIO'] + " " + df_classif['VALIDADOR_TIPO'] + " " + df_classif['VALIDADOR'], inplace=True)
# df_classif['SETOR'] = df_classif['SETOR'].str.replace('LOURDES ITATIAIUCU', 'LOURDES/ITATIAIUCU')
# df_classif['SETOR'] = df_classif['SETOR'].str.replace('CENTRO PADRE EUSTAQUIO', 'CENTRO/PADRE EUSTAQUIO')
# df_classif['SETOR'] = df_classif['SETOR'].str.replace('SANTANENSE MORADA NOVA', 'SANTANENSE/MORADA NOVA')
# df_classif = df_classif.reset_index().drop_duplicates('index').set_index('index')


# def filter_23(df):
#     return df[
#         df['MUNICIPIO'].isin([
#             'DIVINOPOLIS',
#             'ITAUNA',
#             'ITATIAIUCU',
#             'CARMO DO CAJURU',
#             'SAO GONCALO DO PARA',
#             'CLAUDIO'
#         ])
#     ]


# df1 = pd.read_csv('files/RAT/' + rats_files[0], error_bad_lines=False, sep='|')
# df1 = df1.applymap(lambda x: x.strip() if type(x) == str else x)
# df1 = filter_23(df1)

# df2 = pd.read_csv('files/RAT/' + rats_files[1], error_bad_lines=False, sep='|')
# df2 = df2.applymap(lambda x: x.strip() if type(x) == str else x)
# df2 = filter_23(df2)

# df_acum = pd.concat([df1, df2])

# for rat_file in rats_files[2:]:
#     df_aux = pd.read_csv('files/RAT/' + rat_file, error_bad_lines=False, sep='|')
#     df_aux = df_aux.applymap(lambda x: x.strip() if type(x) == str else x)
#     df_aux = filter_23(df_aux)
#     df_acum = pd.concat([df_acum, df_aux])

# df_acum.drop_duplicates(subset='RAT.NUM_ATIVIDADE', keep='last', inplace=True)

# s_dta_in = df_acum['DTA_INICIO'] + " " + df_acum['HRA_INICIO']
# df_acum.loc[:,'DTA_HRA_INICIO_DT'] = pd.to_datetime( s_dta_in, format='%d/%m/%Y %H:%M', errors='coerce')
# del s_dta_in

# s_dta_ter = df_acum.loc[:,'DTA_TERMINO'] + " " + df_acum.loc[:,'HRA_TERMINO']
# df_acum.loc[:,'DTA_HRA_TERMINO_DT'] = pd.to_datetime( s_dta_ter, format='%d/%m/%Y %H:%M', errors='coerce')
# del s_dta_ter

# df_acum.loc[:,'TEMPO_DT'] = df_acum['DTA_HRA_TERMINO_DT'] - df_acum['DTA_HRA_INICIO_DT']

# df_acum.loc[:,'TEMPO_INT'] = df_acum['TEMPO_DT'].dt.total_seconds() / 60

# df_acum.loc[:,'MES'] = df_acum.loc[:,'DTA_HRA_INICIO_DT'].dt.month

# df_acum.loc[:,'ANO'] = df_acum.loc[:,'DTA_HRA_INICIO_DT'].dt.year


# for field in ['TEMPO_INT','ANO','MES']:
#     df_acum[field].fillna(0, inplace=True)
#     df_acum[field] = df_acum[field].astype('int32')

# cols_classif = [
#     'MUNICIPIO',
#     'LOGRADOURO',
#     'DES_ENDERECO',
#     'COMPLEMENTO_ENDERECO',
#     'NOME_BAIRRO',
#     'LOGRADOURO2',
#     'DES_ENDERECO2'
# ]
# df_acum[cols_classif] = df_acum[cols_classif].apply(lambda col: col.str.upper())


# def classifica_setor(row):
#     mun = row['MUNICIPIO']
#     if mun == 'CLAUDIO':        
#         return 'CLAUDIO'
#     elif mun == 'ITATIAIUCU':
#         return 'LOURDES/ITATIAIUCU'
#     elif mun in ('CARMO DO CAJURU', 'SAO GONCALO DO PARA'):             
#         return 'CARMO DO CAJURU/SAO GONCALO DO PARA'    
#     elif ( mun + " N_RAT " + row['RAT.NUM_ATIVIDADE'] ) in df_classif.index:
#         return df_classif.loc[mun+" N_RAT "+row['RAT.NUM_ATIVIDADE'], 'SETOR']
#     elif mun + ' BAIRRO ' + row['NOME_BAIRRO'] in df_classif.index:       
#         return ( df_classif.loc[mun + ' BAIRRO ' + row['NOME_BAIRRO'].upper(), 'SETOR'] ).upper()
#     elif mun + ' LOGRADOURO ' + row['LOGRADOURO'] in df_classif.index:
#         return df_classif.loc[mun + ' LOGRADOURO ' + row['LOGRADOURO'].upper(), 'SETOR']
#     elif mun + ' LOGRADOURO ' + row['DES_ENDERECO'] in df_classif.index:
#         return df_classif.loc[mun + ' LOGRADOURO ' + row['DES_ENDERECO'].upper(), 'SETOR']
#     elif mun + ' LOGRADOURO_NAO_CAD ' + row['LOGRADOURO2'] in df_classif.index:
#         return df_classif.loc[mun + ' LOGRADOURO_NAO_CAD ' + row['LOGRADOURO2'].upper(), 'SETOR']
#     elif mun + ' LOGRADOURO_NAO_CAD ' + row['DES_ENDERECO'] in df_classif.index:
#         return df_classif.loc[mun + ' LOGRADOURO_NAO_CAD ' + row['DES_ENDERECO'].upper(), 'SETOR']
#     elif mun + ' LOGRADOURO_NAO_CAD ' + row['LOGRADOURO2'] in df_classif.index:
#         return df_classif.loc[mun + ' LOGRADOURO_NAO_CAD ' + row['LOGRADOURO2'].upper(), 'SETOR']
#     elif mun + ' LOGRADOURO_NAO_CAD ' + row['DES_ENDERECO2'] in df_classif.index:
#         return df_classif.loc[mun + ' LOGRADOURO_NAO_CAD ' + row['DES_ENDERECO2'].upper(), 'SETOR']
#     elif mun + ' COMPLEMENTO_END ' + row['COMPLEMENTO_ENDERECO'] in df_classif.index:
#         return df_classif.loc[mun + ' COMPLEMENTO_END ' + row['COMPLEMENTO_ENDERECO'].upper(), 'SETOR']
#     elif ( mun + ' COMPLEMENTO_END ' + row['DES_ENDERECO'] ) in df_classif.index:
#         return df_classif.loc[mun + ' COMPLEMENTO_END ' + row['DES_ENDERECO'], 'SETOR']    
#     else:
#         return 'other'

# df_acum['SETOR'] = df_acum.apply(lambda row: classifica_setor(row), axis=1)

# conds=[
#     df_acum['MUNICIPIO'].isin(['ITAUNA','ITATIAIUCU']),
#     df_acum['SETOR'].isin(['HIPER CENTRO','BOM PASTOR','ALTO GOIAS']),
#     df_acum['SETOR'].isin(['PLANALTO','SAO JOSE','CLAUDIO']),
#     df_acum['SETOR'].isin(['NITEROI','PORTO VELHO','CARMO DO CAJURU/SAO GONCALO DO PARA']),
    
# ]
# res=['51 CIA','53 CIA','139 CIA','142 CIA']
# df_acum['CIA'] = np.select(conds,res,default='other')

# df_acum.to_sql('tbl_rat', 'sqlite:///gdo.db', if_exists='replace', index=False)
# del df_acum

b'Skipping line 34161: expected 32 fields, saw 33\n'
b'Skipping line 15568: expected 32 fields, saw 33\n'
b'Skipping line 2860: expected 32 fields, saw 33\n'
b'Skipping line 12419: expected 32 fields, saw 33\n'
/home/seo23bpm/.asdf/installs/python/anaconda3-2019.10/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
b'Skipping line 50881: expected 32 fields, saw 33\n'
b'Skipping line 58502: expected 32 fields, saw 33\n'
b'Skipping line 12496: expected 32 fields, saw 33\n'
b'Skipping line 18880: expected 32 fields, saw 33\nSkipping line 28767: expected 32 fields, saw 37\n'
b'Skipping line 33410: expected 32 fields, saw 33\n'
b'Skipping line 4115: expected 32 fields, saw 33\nSkipping line 10843: expected 32 fields, saw 33\nSkipping line 10898: expected 32 fields, saw 33\n'
b'Skipping line 29942: expected 32 fi

<h2 style="color:#0A0;margin:10px ">Importação, para o Banco de Dados, de todos os arquivos tipo RAT_VEICULOS, RAT_VIATURA e RAT_PRODUTIVIDADE</h2>
<h3 style="color:#00A; margin:10px">Importa somente os dados do 23º BPM</h3>

<ol style="color:#222">
    <li>Importa do Banco de Dados, a tabela rat, para o df_rat, atribuindo RAT.NUM_ATIVIDADE para o index</li>
    <li>Lê o primeiro arquivo da lista e o passa para um DataFrame (dfv1, dfe1, dfp1)</li>
    <li>Lê o segundo arquivo da lista e o passa para outro DataFrame (dfv2, dfe2, dfp2)</li>
    <li>Concatena os dois DataFrames em um terceiro (df_acum)</li>
    <li>Itera sobre os demais arquivos, lendo cada um deles, aplicando strip(), a função de filtro do 23º BPM e depois o adiciona ao DataFrame df_acum</li>    
    <li>Insere os dados no banco de dados</li>        
</ol>    

In [27]:
df_ratv = pd.read_sql_table('tbl_rat', 'sqlite:///gdo.db')
df_ratv.head()

,RAT.NUM_ATIVIDADE,NAT.CODIGO,NAT.DESCRICAO,DTA_HRA_INCLUSAO,DTA_INICIO,HRA_INICIO,DTA_TERMINO,HRA_TERMINO,DES_ALVO_EVENTO,DES_LUGAR,...,NOM_UNIDADE_AREA,DIGITADOR,DTA_HRA_INICIO_DT,DTA_HRA_TERMINO_DT,TEMPO_DT,TEMPO_INT,MES,ANO,SETOR,CIA
0,2018-046472984-001,Y07001,OPERACAO DE BATIDA POLICIAL,17/10/2018,17/10/2018,18:43,17/10/2018,19:00,TRANSEUNTE,VIA DE ACESSO PUBLICA,...,53 CIA PM/23 BPM/7 RPM,PM1569110,2018-10-17 18:43:00,2018-10-17 19:00:00,1.020000e+12,17,10,2018,PLANALTO,139 CIA
1,2018-046878708-001,Y12001,FISCALIZACAO DE LOCAIS DE DESMATE,20/10/2018,20/10/2018,12:39,20/10/2018,15:00,FAZENDA,FAZENDA,...,3 PEL/139 CIA PM/23 BPM/7 RPM,PM1502681,2018-10-20 12:39:00,2018-10-20 15:00:00,8.460000e+12,141,10,2018,CLAUDIO,139 CIA
2,2018-047006854-001,Y11006,FISCALIZACAO DE MAUS TRATOS A ANIMAIS,21/10/2018,21/10/2018,13:28,21/10/2018,16:00,FAZENDA,FAZENDA,...,142 CIA PM/23 BPM/7 RPM,PM1502681,2018-10-21 13:28:00,2018-10-21 16:00:00,9.120000e+12,152,10,2018,PORTO VELHO,142 CIA
3,2018-048663517-001,Y12001,FISCALIZACAO DE LOCAIS DE DESMATE,31/10/2018,31/10/2018,07:23,31/10/2018,10:00,FAZENDA,FAZENDA,...,53 CIA PM/23 BPM/7 RPM,PM1502681,2018-10-31 07:23:00,2018-10-31 10:00:00,9.420000e+12,157,10,2018,PLANALTO,139 CIA
4,2018-049254756-001,Y08999,OUTRAS ESCOLTAS,04/11/2018,04/11/2018,08:22,04/11/2018,10:00,CARGAS,VIA DE ACESSO PUBLICA,...,53 CIA PM/23 BPM/7 RPM,PM1326123,2018-11-04 08:22:00,2018-11-04 10:00:00,5.880000e+12,98,11,2018,BOM PASTOR,53 CIA


In [7]:
# df_rat = pd.read_sql_table('tbl_rat', 'sqlite:///gdo.db')
# df_rat.set_index('RAT.NUM_ATIVIDADE', inplace=True)

# dfv1 = pd.read_csv('files/RAT/' + rats_viatura_files[0], error_bad_lines=False, sep='|')
# dfv1 = dfv1.applymap(lambda x: x.strip() if type(x) == str else x)
# dfv1 = dfv1[dfv1['NUM_ATIVIDADE'].isin(df_rat.index)]

# dfv2 = pd.read_csv('files/RAT/' + rats_viatura_files[1], error_bad_lines=False, sep='|')
# dfv2 = dfv2.applymap(lambda x: x.strip() if type(x) == str else x)
# dfv2 = dfv2[dfv2['NUM_ATIVIDADE'].isin(df_rat.index)]

# dfv_acum = pd.concat([dfv1, dfv2])


# for rat_viatura_file in rats_viatura_files[2:]:
#     dfv_aux = pd.read_csv('files/RAT/'+rat_viatura_file, sep='|')
#     dfv_aux = dfv_aux.applymap(lambda col: col.strip() if type(col) == str else col)
#     dfv_aux = dfv_aux[dfv_aux['NUM_ATIVIDADE'].isin(df_rat.index)]
#     dfv_acum = pd.concat([dfv_acum, dfv_aux])
    
# dfv_acum.to_sql('tbl_rat_veiculo', 'sqlite:///gdo.db', if_exists='replace', index=False)
# del dfv_acum
# del df_rat

In [9]:
# df_rat = pd.read_sql_table('tbl_rat', 'sqlite:///gdo.db')
# df_rat.set_index('RAT.NUM_ATIVIDADE', inplace=True)

# dfe1 = pd.read_csv('files/RAT/' + rats_efetivo_files[0], error_bad_lines=False, sep='|')
# dfe1 = dfe1.applymap(lambda x: x.strip() if type(x) == str else x)
# dfe1 = dfe1[dfe1['NUM_ATIVIDADE'].isin(df_rat.index)]

# dfe2 = pd.read_csv('files/RAT/' + rats_efetivo_files[1], error_bad_lines=False, sep='|')
# dfe2 = dfe2.applymap(lambda x: x.strip() if type(x) == str else x)
# dfe2 = dfe2[dfe2['NUM_ATIVIDADE'].isin(df_rat.index)]

# dfe_acum = pd.concat([dfe1, dfe2])


# for rat_efetivo_file in rats_efetivo_files[2:]:
#     dfe_aux = pd.read_csv('files/RAT/'+rat_efetivo_file, sep='|')
#     dfe_aux = dfe_aux.applymap(lambda col: col.strip() if type(col) == str else col)
#     dfe_aux = dfe_aux[dfe_aux['NUM_ATIVIDADE'].isin(df_rat.index)]
#     dfe_acum = pd.concat([dfe_acum, dfe_aux])
    
# dfe_acum.to_sql('tbl_rat_efetivo', 'sqlite:///gdo.db', if_exists='replace', index=False)
# del dfe_acum
# del df_rat

In [10]:
# df_rat = pd.read_sql_table('tbl_rat', 'sqlite:///gdo.db')
# df_rat.set_index('RAT.NUM_ATIVIDADE', inplace=True)

# dfp1 = pd.read_csv('files/RAT/' + rats_produtividade_files[0], error_bad_lines=False, sep='|')
# dfp1 = dfp1.applymap(lambda x: x.strip() if type(x) == str else x)
# dfp1 = dfp1[dfp1['RAT.NUM_ATIVIDADE'].isin(df_rat.index)]

# dfp2 = pd.read_csv('files/RAT/' + rats_produtividade_files[1], error_bad_lines=False, sep='|')
# dfp2 = dfp2.applymap(lambda x: x.strip() if type(x) == str else x)
# dfp2 = dfp2[dfp2['RAT.NUM_ATIVIDADE'].isin(df_rat.index)]

# dfp_acum = pd.concat([dfp1, dfp2])


# for rat_produtividade_file in rats_produtividade_files[2:]:
#     dfp_aux = pd.read_csv('files/RAT/'+rat_produtividade_file, sep='|')
#     dfp_aux = dfp_aux.applymap(lambda col: col.strip() if type(col) == str else col)
#     dfp_aux = dfp_aux[dfp_aux['RAT.NUM_ATIVIDADE'].isin(df_rat.index)]
#     dfp_acum = pd.concat([dfp_acum, dfp_aux])
    
# dfp_acum.to_sql('tbl_rat_produtividade', 'sqlite:///gdo.db', if_exists='replace', index=False)
# del df_rat
# del dfp_acum